# NER Named Entity Recognition

In this notebook I did some analysis in which I used Named Entity Recognition technique and also **fine-tuning** approach with using available model **"biobert"** to identify medical names in given text from clinical trials.  

# Data import and initial analysis

In [ ]:
import pandas as pd
df_selected_columns = pd.read_csv('df_selected_columns.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_selected_columns.shape

(4000, 54)

In [ ]:
df_selected_columns.head()

,Unnamed: 0,hasResults,identificationModule.nctId,identificationModule.organization.fullName,identificationModule.organization.class,identificationModule.briefTitle,identificationModule.officialTitle,statusModule.statusVerifiedDate,statusModule.studyFirstSubmitDate,statusModule.studyFirstPostDateStruct.type,statusModule.lastUpdatePostDateStruct.date,sponsorCollaboratorsModule.responsibleParty.type,sponsorCollaboratorsModule.leadSponsor.name,sponsorCollaboratorsModule.leadSponsor.class,descriptionModule.briefSummary,descriptionModule.detailedDescription,designModule.studyType,designModule.enrollmentInfo.count,designModule.enrollmentInfo.type,eligibilityModule.eligibilityCriteria,eligibilityModule.sex,eligibilityModule.minimumAge,armsInterventionsModule.armGroups_0_label,armsInterventionsModule.armGroups_0_description,armsInterventionsModule.armGroups_0_type,armsInterventionsModule.armGroups_1_label,armsInterventionsModule.armGroups_1_description,armsInterventionsModule.armGroups_1_type,armsInterventionsModule.interventions_0_type,armsInterventionsModule.interventions_0_name,armsInterventionsModule.interventions_0_description,outcomesModule.primaryOutcomes_0_measure,outcomesModule.primaryOutcomes_0_description,outcomesModule.primaryOutcomes_0_timeFrame,outcomesModule.secondaryOutcomes_0_measure,outcomesModule.secondaryOutcomes_0_description,outcomesModule.secondaryOutcomes_0_timeFrame,contactsLocationsModule.overallOfficials_0_name,contactsLocationsModule.overallOfficials_0_affiliation,contactsLocationsModule.overallOfficials_0_role,contactsLocationsModule.locations_0_facility,contactsLocationsModule.locations_0_city,contactsLocationsModule.locations_0_country,conditionBrowseModule.meshes_0_term,conditionBrowseModule.ancestors_0_term,conditionBrowseModule.ancestors_1_term,conditionBrowseModule.ancestors_2_term,conditionBrowseModule.browseLeaves_0_name,conditionBrowseModule.browseLeaves_1_name,conditionBrowseModule.browseLeaves_2_name,conditionBrowseModule.browseLeaves_3_name,conditionBrowseModule.browseLeaves_4_name,conditionBrowseModule.browseBranches_0_name,conditionBrowseModule.browseBranches_2_name
0,0,False,nct06733961,giresun university,NaN,scube 1 vap 1 levels diagnosis pulmonary embolism,investigation relationship scube 1 vap 1 level...,2024 11,2024 12 01,estimated,2024 12 13,principal_investigator,giresun university,NaN,acute pulmonary thromboembolism life threateni...,definitive diagnosis apt made ventilation perf...,observational,88,actual,inclusion criteria older 18 years age presenti...,NaN,18 years,patient group,consists patients diagnosed pulmonary thromboe...,NaN,control group,composed healthy volunteers similar age gender...,NaN,diagnostic_test,scube 1 vap 1,study comparing biomarkers healthy volunteers ...,diagnostic ability scube 1 vap 1 pulmonary emb...,serum scube 1 serum vap 1 levels patient contr...,6 month,difference clinical probability scores serum s...,clinical probability scores patient group grou...,6 month,hizir u akdemir prof,ondokuz mayıs university,study_chair,ondokuz mayis university,samsun,turkey,pulmonary embolism,embolism thrombosis,vascular diseases,cardiovascular diseases,pulmonary embolism,embolism,thromboembolism,emergencies,thrombosis,respiratory tract lung bronchial diseases,conditions
1,1,False,nct06733857,game changers institute,NaN,environmental costs building human muscle,environmental costs building human muscle comp...,2024 12,2024 12 04,estimated,2024 12 13,sponsor,game changers institute,NaN,among general population established plant bas...,development skeletal muscle mass associated im...,interventional,22,actual,inclusion criteria males females 18 40 years a...,NaN,18 years,plant based,participants consume high protein plant based ...,experimental,omnivorous,participants consume high protein omnivorous d...,active_comparator,behavioral,veg2,participants provided caloric target place 0 1...,diet related greenhouse gas emissions co2 equi...,diet related greenhouse gas emissions 

In [ ]:
df_selected_columns.isna().sum()

,0
Unnamed: 0,0
hasResults,0
identificationModule.nctId,0
identificationModule.organization.fullName,0
identificationModule.organization.class,3324
identificationModule.briefTitle,0
identificationModule.officialTitle,1
statusModule.statusVerifiedDate,0
statusModule.studyFirstSubmitDate,0
statusModule.studyFirstPostDateStruct.type,0


In [ ]:
df_selected_columns = df_selected_columns.drop(columns=['identificationModule.organization.class','sponsorCollaboratorsModule.leadSponsor.class','outcomesModule.secondaryOutcomes_0_measure'])

In [ ]:
df_selected_columns = df_selected_columns.loc[:, df_selected_columns.isna().sum() <= 200]

In [ ]:
# Filling missing values in text columns with empty strings
df_selected_columns_2 = df_selected_columns.copy()
for col in df_selected_columns.select_dtypes(include=['object']).columns:
    df_selected_columns[col].fillna("", inplace=True)

<ipython-input-9-736e8da4c115>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_selected_columns[col].fillna("", inplace=True)


In [ ]:
df_selected_columns.isna().sum()

,0
Unnamed: 0,0
hasResults,0
identificationModule.nctId,0
identificationModule.organization.fullName,0
identificationModule.briefTitle,0
identificationModule.officialTitle,0
statusModule.statusVerifiedDate,0
statusModule.studyFirstSubmitDate,0
statusModule.studyFirstPostDateStruct.type,0
statusModule.lastUpdatePostDateStruct.date,0


In [ ]:
# Deleting 'identificationModule.nctId'
df_selected_columns = df_selected_columns.drop(columns=['identificationModule.nctId'])

In [ ]:
df_selected_columns = df_selected_columns.loc[:, df_selected_columns.isna().sum() <= 100]

In [ ]:
df_selected_columns.shape

(4000, 21)

In [ ]:
df_selected_columns.nunique()

,0
Unnamed: 0,4000
hasResults,2
identificationModule.organization.fullName,1722
identificationModule.briefTitle,4000
identificationModule.officialTitle,3994
statusModule.statusVerifiedDate,63
statusModule.studyFirstSubmitDate,447
statusModule.studyFirstPostDateStruct.type,2
statusModule.lastUpdatePostDateStruct.date,139
sponsorCollaboratorsModule.responsibleParty.type,3


In [ ]:
selected_columns = [
    "identificationModule.briefTitle",
    "identificationModule.officialTitle",
]

In [ ]:
df_selected = df_selected_columns[selected_columns]


In [ ]:
df_selected.shape

(4000, 2)

In [ ]:
df_selected.head()

,identificationModule.briefTitle,identificationModule.officialTitle
0,scube 1 vap 1 levels diagnosis pulmonary embolism,investigation relationship scube 1 vap 1 level...
1,environmental costs building human muscle,environmental costs building human muscle comp...
2,physical function perspective active ageing,evaluation levels physical function older adul...
3,investigation analysis nis protection level in...,investigation analysis nis protection level in...
4,study evaluate oral formulations tedizolid pho...,study evaluate pharmacokinetics definitive bio...


# Fine-tuning approach

In this part, I am using the **BioBERT** (Biomedical Bidirectional Encoder Representations from Transformers) model to fine-tune it with my data (clinical trials).

For this exercise I will use only two columns (with brief Title and official title)


In [ ]:
def combine_columns(row):
    combined_text = (
        f"Brief Title: {row['identificationModule.briefTitle']} "
        f"Official Title: {row['identificationModule.officialTitle']} "
    )
    return combined_text

df_selected['combined_text'] = df_selected.apply(combine_columns, axis=1)

<ipython-input-14-7c47e1f1c44e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['combined_text'] = df_selected.apply(combine_columns, axis=1)


In [ ]:
df_selected.head()

,identificationModule.briefTitle,identificationModule.officialTitle,combined_text
0,scube 1 vap 1 levels diagnosis pulmonary embolism,investigation relationship scube 1 vap 1 level...,Brief Title: scube 1 vap 1 levels diagnosis pu...
1,environmental costs building human muscle,environmental costs building human muscle comp...,Brief Title: environmental costs building huma...
2,physical function perspective active ageing,evaluation levels physical function older adul...,Brief Title: physical function perspective act...
3,investigation analysis nis protection level in...,investigation analysis nis protection level in...,Brief Title: investigation analysis nis protec...
4,study evaluate oral formulations tedizolid pho...,study evaluate pharmacokinetics definitive bio...,Brief Title: study evaluate oral formulations ...


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.



## Tokenization
Every word in the given text will be converted into a token and assigned a special label:

*  B-DISEASE (Beginning): Indicates the beginning of a recognized entity (e.g., the name of a disease).
*  I-DISEASE (Inside): Indicates subsequent tokens that belong to the same entity.
*  O (Outside): Indicates that the token is not part of any recognized entity.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset


In [ ]:
from transformers import pipeline, AutoTokenizer
import pandas as pd

# Model and tokenizer loading
model_name = "dmis-lab/biobert-v1.1"
nlp_ner = pipeline("ner", model=model_name, tokenizer=model_name, aggregation_strategy="simple")
tokenizer = AutoTokenizer.from_pretrained(model_name)

#function for generating labels
def generate_labels(text):
    # Tokenization of text
    tokens = tokenizer.tokenize(text)
    labels = ['O'] * len(tokens)  # Domyślna etykieta to 'O'

    # Identifying medical entities thanks to NER
    entities = nlp_ner(text)

    for entity in entities:
        entity_text = entity['word'].replace("##", "")
        entity_tokens = tokenizer.tokenize(entity_text)

        # Searching for token's indexes in the text
        for i, token in enumerate(entity_tokens):
            try:
                token_idx = tokens.index(token)
                if i == 0:
                    labels[token_idx] = 'B-DISEASE'
                else:
                    labels[token_idx] = 'I-DISEASE'
            except ValueError:
                continue

    return labels

# Assigning labels for every row
df_selected['labels'] = df_selected['combined_text'].apply(generate_labels)


print(df_selected[['combined_text', 'labels']])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


                                          combined_text  \
0     Brief Title: scube 1 vap 1 levels diagnosis pu...   
1     Brief Title: environmental costs building huma...   
2     Brief Title: physical function perspective act...   
3     Brief Title: investigation analysis nis protec...   
4     Brief Title: study evaluate oral formulations ...   
...                                                 ...   
2855  Brief Title: feasibility acceptability fidelit...   
2856  Brief Title: dao deficiency screening among pa...   
2857  Brief Title: valsartan amlodipine blood pressu...   
2858  Brief Title: knowledge awareness pelvic floor ...   
2859  Brief Title: study learn safe bay3283142 taken...   

                                                 labels  
0     [B-DISEASE, I-DISEASE, B-DISEASE, I-DISEASE, I...  
1     [B-DISEASE, I-DISEASE, I-DISEASE, I-DISEASE, I...  
2     [B-DISEASE, I-DISEASE, I-DISEASE, I-DISEASE, I...  
3     [B-DISEASE, I-DISEASE, B-DISEASE, I-DISEASE, B...  
4

<ipython-input-18-419e2fb77c3d>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['labels'] = df_selected['combined_text'].apply(generate_labels)


below code implements a complete pipeline for token classification in clinical texts using the BioBERT model – from data preparation to model training and evaluation.

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_scheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tqdm import tqdm


model_name = "dmis-lab/biobert-v1.1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
num_epochs = 3
learning_rate = 5e-5
max_length = 128


tokenizer = AutoTokenizer.from_pretrained(model_name)

#this class is for converting text labels to numeric and tokenize text
class ClinicalTrialsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length, label_map):
        self.texts = texts
        self.labels = [[label_map[label] for label in label_seq] for label_seq in labels]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        labels = self.labels[idx][:self.max_length] + [-100] * (self.max_length - len(self.labels[idx][:self.max_length]))
        return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0), torch.tensor(labels)

# Data preparation
# Using data with labels
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_selected["combined_text"].tolist(),
    df_selected["labels"].tolist(),
    test_size=0.2,
    random_state=42
)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42
)

# Mapping labels
all_labels = set(label for label_seq in df_selected["labels"].tolist() for label in label_seq)
label_map = {label: idx for idx, label in enumerate(all_labels)}
print("Label Map:", label_map)

# Datasets creation
train_dataset = ClinicalTrialsDataset(train_texts, train_labels, tokenizer, max_length, label_map)
val_dataset = ClinicalTrialsDataset(val_texts, val_labels, tokenizer, max_length, label_map)
test_dataset = ClinicalTrialsDataset(test_texts, test_labels, tokenizer, max_length, label_map)

# unction for loading batches that ensures token sequences and labels are properly padded.
def custom_collate_fn(batch):
    input_ids, attention_masks, labels = zip(*batch)
    padded_input_ids = torch.stack([
        torch.cat([input_id[:max_length], torch.full((max_length - len(input_id[:max_length]),), tokenizer.pad_token_id)])
        for input_id in input_ids
    ])
    padded_attention_masks = torch.stack([
        torch.cat([attention_mask[:max_length], torch.zeros(max_length - len(attention_mask[:max_length]))])
        for attention_mask in attention_masks
    ])
    padded_labels = torch.stack([
        torch.cat([label[:max_length], torch.full((max_length - len(label[:max_length]),), -100)])
        for label in labels
    ])
    return padded_input_ids, padded_attention_masks, padded_labels

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)

# Model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_map))
model.to(device)

# Optimizer i scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs)

def train_epoch(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluation"):
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return total_loss / len(dataloader), predictions, true_labels


for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler)
    val_loss, _, _ = evaluate(model, val_dataloader)
    print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")


def flat_list(predictions, labels):
    flat_preds, flat_labels = [], []
    for pred, label in zip(predictions, labels):
        for p, l in zip(pred, label):
            if l != -100:
                flat_preds.append(p)
                flat_labels.append(l)
    return flat_preds, flat_labels

_, test_predictions, test_labels = evaluate(model, test_dataloader)
predicted_labels, true_labels = flat_list(test_predictions, test_labels)

print("Classification Report:\n", classification_report(true_labels, predicted_labels))
print("Confusion Matrix:\n", confusion_matrix(true_labels, predicted_labels))




Label Map: {'I-DISEASE': 0, 'O': 1, 'B-DISEASE': 2}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Evaluation: 100%|██████████| 58/58 [02:59<00:00,  3.10s/it]


Training Loss: 0.3771, Validation Loss: 0.2903
Epoch 2/3


...

Training Loss: 0.2530, Validation Loss: 0.2739
Epoch 3/3


Evaluation: 100%|██████████| 58/58 [03:01<00:00,  3.13s/it]


Training Loss: 0.2055, Validation Loss: 0.2795


Evaluation: 100%|██████████| 72/72 [03:50<00:00,  3.21s/it]

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.93      0.88     11556
           1       0.96      0.93      0.95     10730
           2       0.75      0.51      0.61      2687

    accuracy                           0.88     24973
   macro avg       0.85      0.79      0.81     24973
weighted avg       0.88      0.88      0.88     24973

Confusion Matrix:
 [[10705   394   457]
 [  728 10001     1]
 [ 1314     8  1365]]


## summary  of classification report and confusion matrix
In general we can observe that model is learning quite good on training data - training loss is decreasing in every epoch.
*   fine-tuned model is recognizing very well tokens "0" (with label = 1) with high precision and recall, f1 score is 95%
*   similarly for tokens "I-Disease" (with label = 0)  where f1 score is 88%  
*   the worst f1 score is for "B-Disease" (with label = 2) which is 61%, with recall = 51% may means that model is loosing begining of words
* 457 cases of I-DISEASE were wrongly classified as B-DISEASE
*  1,314 instances of B-DISEASE were misclassified as I-DISEASE, indicating that the model is losing information about the beginning of the entity



## suggestions for improvements
*  Increasing data for the B-DISEASE class – the model struggles to recognize the beginnings of entities, which may be due to an insufficient number of examples in the training data.
*  Applying regularization techniques – to minimize overfitting.
*  Experimenting with hyperparameters – e.g., reducing the number of epochs or introducing early stopping.
*  Better loss function adjustment – e.g., applying class weights in the loss function to balance the impact of rare classes.

## saving and loading trained model

In [ ]:
import os

output_dir = "./saved_model"


os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


label_map_path = os.path.join(output_dir, "label_map.json")
import json
with open(label_map_path, "w") as f:
    json.dump(label_map, f)

print(f"Model, tokenizer, and label map saved to {output_dir}")


Model, tokenizer, and label map saved to ./saved_model


In [ ]:
from transformers import BertForTokenClassification, BertTokenizer


model_path = "/content/model"

tokenizer = BertTokenizer.from_pretrained(model_path)


model = BertForTokenClassification.from_pretrained(model_path)


In [ ]:
df_selected['combined_text'] = df_selected['identificationModule.briefTitle'] + ' ' + df_selected['identificationModule.officialTitle']

<ipython-input-19-b06d23a61355>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['combined_text'] = df_selected['identificationModule.briefTitle'] + ' ' + df_selected['identificationModule.officialTitle']


In [ ]:
from transformers import pipeline
nlp_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cpu


## analysis of clinical text with trained model
Below code applies a Named Entity Recognition (NER) model to a DataFrame containing clinical text.
In  the first step we extracted labels with scores (The closer the score is to 1, the more confident the model is in its prediction), example: *[{'entity_group': 'DISEASE', 'word': 'diabetes', 'score': 0.98}] *





In [ ]:

def analyze_entities(text):
    results = nlp_pipeline(text)
    return results


df_selected['entities'] = df_selected['combined_text'].apply(analyze_entities)

print(df_selected[['combined_text', 'entities']].head())


                                       combined_text  \
0  scube 1 vap 1 levels diagnosis pulmonary embol...   
1  environmental costs building human muscle envi...   
2  physical function perspective active ageing ev...   
3  investigation analysis nis protection level in...   
4  study evaluate oral formulations tedizolid pho...   

                                            entities  
0  [{'entity_group': 'LABEL_0', 'score': 0.829086...  
1  [{'entity_group': 'LABEL_0', 'score': 0.731408...  
2  [{'entity_group': 'LABEL_0', 'score': 0.922636...  
3  [{'entity_group': 'LABEL_2', 'score': 0.612218...  
4  [{'entity_group': 'LABEL_0', 'score': 0.788585...  


<ipython-input-23-014d9a996317>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['entities'] = df_selected['combined_text'].apply(analyze_entities)


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

def extract_entities(text):
    return nlp_pipeline(text)


df_selected['medical_entities'] = df_selected['combined_text'].apply(extract_entities)


print(df_selected[['combined_text', 'medical_entities']].head())


                                       combined_text  \
0  scube 1 vap 1 levels diagnosis pulmonary embol...   
1  environmental costs building human muscle envi...   
2  physical function perspective active ageing ev...   
3  investigation analysis nis protection level in...   
4  study evaluate oral formulations tedizolid pho...   

                                    medical_entities  
0  [{'entity_group': 'LABEL_0', 'score': 0.829086...  
1  [{'entity_group': 'LABEL_0', 'score': 0.731408...  
2  [{'entity_group': 'LABEL_0', 'score': 0.922636...  
3  [{'entity_group': 'LABEL_2', 'score': 0.612218...  
4  [{'entity_group': 'LABEL_0', 'score': 0.788585...  


<ipython-input-25-b8ba903a3433>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['medical_entities'] = df_selected['combined_text'].apply(extract_entities)


In [ ]:
print(df_selected[['combined_text', 'medical_entities']])

                                          combined_text  \
0     scube 1 vap 1 levels diagnosis pulmonary embol...   
1     environmental costs building human muscle envi...   
2     physical function perspective active ageing ev...   
3     investigation analysis nis protection level in...   
4     study evaluate oral formulations tedizolid pho...   
...                                                 ...   
3995  validating wireless gait sensor elderly fall r...   
3996  efficacy proinflammatory cytokines primary mol...   
3997  effect deep breathing balance patients chronic...   
3998  effect visuomotor training using pablo system ...   
3999  novasure expierences novex novasure endometria...   

                                       medical_entities  
0     [{'entity_group': 'LABEL_0', 'score': 0.829086...  
1     [{'entity_group': 'LABEL_0', 'score': 0.731408...  
2     [{'entity_group': 'LABEL_0', 'score': 0.922636...  
3     [{'entity_group': 'LABEL_2', 'score': 0.612218...  
4

In the next step we are going to convert a list of entity dictionaries into a human-readable string format

In [ ]:

def format_entities(entities):
    return "; ".join([f"{e['entity_group']} ({e['word']})" for e in entities])


df_selected['formatted_entities'] = df_selected['medical_entities'].apply(format_entities)


<ipython-input-29-8e50c8982ac4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['formatted_entities'] = df_selected['medical_entities'].apply(format_entities)


In [ ]:
# csv export
df_selected[['combined_text', 'formatted_entities']].to_csv("medical_entities_analysis.csv", index=False, encoding='utf-8')

print("Plik CSV został zapisany jako 'medical_entities_analysis.csv'")


Plik CSV został zapisany jako 'medical_entities_analysis.csv'


In [ ]:
from collections import Counter

# counter
entity_counts = Counter([entity['entity_group'] for row in df_selected['entities'] for entity in row])
print(entity_counts)


Counter({'LABEL_0': 15723, 'LABEL_1': 13332, 'LABEL_2': 7184})


In general we can see that model is indentyfing LABEL_2 which stands for B-Disease tokens (the beginnings of medical entities) with 7184 medical names.  

## possible next steps
With the named entities already identified, several analyses can be performed to gain deeper insights:
*   Topic Modeling: Apply methods like LDA or BERTopic to discover hidden topics based on the identified entities.
*   Entity Linking: Enrich the data by connecting recognized entities to external databases (e.g., PubMed, ClinicalTrials.gov) for additional context.
*   Document Classification: Use the identified entities as features to classify clinical trials, such as by intervention type or trial phase.



# NER analysis (using external language model )
We can also try to analyse clinical trials only by using already trained model for processing text and recognizing entities.  

In [ ]:
import pandas as pd
import spacy

# loading language model
nlp = spacy.load("en_core_web_sm")

# Combining Text Columns
texts = df_selected_columns.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Performing Named Entity Recognition
all_entities = []
for i, doc in enumerate(nlp.pipe(texts, disable=["tagger", "parser"])):  # pipe przyspiesza przetwarzanie dużych danych
    for ent in doc.ents:
        all_entities.append([i, ent.text, ent.label_])  # dodaj indeks dokumentu

# Storing the Results
entities_df = pd.DataFrame(all_entities, columns=["Document_Index", "Entity", "Label"])
print(entities_df.head())

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


   Document_Index Entity     Label
0               0      0  CARDINAL
1               0      1  CARDINAL
2               0      1  CARDINAL
3               0      1  CARDINAL
4               0      1  CARDINAL


In [ ]:
entity_counts = entities_df['Label'].value_counts()
print(entity_counts)

Label
CARDINAL       32504
DATE           24409
GPE             5742
ORG             4797
PERSON          3147
QUANTITY        2212
TIME            1858
ORDINAL         1686
NORP            1627
PRODUCT          216
LOC              166
FAC              142
MONEY            115
LANGUAGE          66
LAW               43
EVENT             32
PERCENT           29
WORK_OF_ART        5
Name: count, dtype: int64


In [ ]:
money_entities = entities_df[entities_df["Label"] == "MONEY"]

unique_money_entities = money_entities["Entity"].unique()
print("Unikalne encje typu 'MONEY':")
print(unique_money_entities)



Unikalne encje typu 'MONEY':
['12 04 estimated' '13' '152 8 million' '08 13 estimated' '29 estimated'
 '005' '09' '06' '05' '170 False' '04' 'ejection fraction iron'
 '110 pounds' '356 False gazi' '2' '446 False' '532 False menoufia'
 '555 False gaziosmanpasa' '578 False' '19 pneumonia'
 '621 False corestemchemon inc evaluate' '622 False okan university'
 '731 False noom inc longitudinal' '31 sponsor noom inc'
 '778 False islamic azad university' '5l' '795 False ataturk' '70'
 '870 False san raffaele' '897 False islamic azad university'
 '934 False bakirkoy' '956 False fondation' '201' '11' 'millions dollars'
 '028' '500 ml' '007' '10 per cent' '0 biotrial inc newark'
 '18months tokat gaziosmanpasa' '12 tanta university tanta' '100 ml day'
 '29 principal_investigator' '2211 False gaziosmanpasa' '1 2 billion'
 '48hours sohag' '30' '25' '321' '2b' '3l' '1 celerion inc' 'dollar penny'
 '3 yaroslavl russian federation' '1 neurogenic congenial'
 '14 identity inc gaithersburg' '08' '35 low o

In [ ]:
cardinal_entities = entities_df[entities_df["Label"] == "CARDINAL"]

unique_cardinal_entities = cardinal_entities["Entity"].unique()
print("Unikalne encje typu 'CARDINAL':")
print(unique_cardinal_entities)

Unikalne encje typu 'CARDINAL':
['0' '1' '12' ... '3997' '3998' 'approximately one']


In [ ]:
org_entities = entities_df[entities_df["Label"] == "ORG"]

org_cardinal_entities = org_entities["Entity"].unique()
print("Unikalne encje typu 'ORG':")
print(org_cardinal_entities)

Unikalne encje typu 'ORG':
['giresun university acute pulmonary' 'ibm' 'acute pte' ...
 'data national health nutrition examination'
 'national university yangsan' 'gait data']


In [ ]:
loc_entities = entities_df[entities_df["Label"] == "LOC"]


unique_loc_entities = loc_entities["Entity"].value_counts()
print("Występowanie LOC':")
print(unique_loc_entities)

Występowanie LOC':
Entity
west china           19
spina bifida         13
asia                 10
africa               10
europe                9
                     ..
atlantic              1
xingtai china         1
city heilongjiang     1
southwest china       1
postdam area          1
Name: count, Length: 72, dtype: int64


In [ ]:
gpe_entities = entities_df[entities_df["Label"] == "GPE"]

unique_gpe_entities = gpe_entities["Entity"].value_counts()
print("Występowanie GPE':")
print(unique_gpe_entities)

Występowanie GPE':
Entity
china                      503
egypt                      460
anesthesia                 404
cairo                      278
united states              240
                          ... 
ann arbor united states      1
el behira                    1
british                      1
de mexico mexico             1
hefei city                   1
Name: count, Length: 655, dtype: int64


# Summary

NER gives few options for further analysing clinical data. It looks like **NER with extrenal model like "en_core_web_sm"** is able to provide fast exploratory data analysis. However if we are looking for more detailed insights especially in medical data then **Fine-Tuning with BioBert** is much more helpful, although we have to keep in mind that we need to have labeled data, it is also more complex and there is risk that we may overfit data.
